# Forbered Data 2

In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from scipy.stats import mode
from sklearn.neighbors import KNeighborsClassifier
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge

pd.options.display.max_columns = 1000
pd.options.display.max_info_columns = 1000
pd.options.display.max_info_rows = 1000
pd.options.display.max_rows = 50


In [2]:
pd.read_csv('data/sample_submission.csv')

,id,outcome
0,1235,lived
1,1236,lived
2,1237,lived
3,1238,lived
4,1239,lived
...,...,...
819,2054,lived
820,2055,lived
821,2056,lived
822,2057,lived


In [3]:
data = pd.read_csv('data/train.csv')

data

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,NaN,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1230,yes,adult,535246,38.5,129.0,48.0,cool,reduced,pale_pink,more_3_sec,depressed,absent,moderate,none,more_1_liter,2.0,NaN,distend_large,57.0,66.0,serosanguious,2.0,yes,2206,0,0,no,lived
1231,1231,yes,adult,528570,37.5,60.0,50.0,cool,reduced,pale_cyanotic,less_3_sec,mild_pain,hypomotile,slight,slight,none,3.0,decreased,distend_small,35.0,6.4,serosanguious,3.6,yes,2209,0,0,yes,died
1232,1232,yes,young,529685,37.5,84.0,40.0,normal,reduced,normal_pink,less_3_sec,mild_pain,hypomotile,slight,slight,none,3.0,increased,firm,40.0,5.9,cloudy,7.0,yes,400,0,0,yes,lived
1233,1233,yes,adult,534784,38.1,70.0,16.0,normal,reduced,bright_red,less_3_sec,mild_pain,hypomotile,slight,none,more_1_liter,2.0,absent,distend_small,58.0,74.0,cloudy,2.0,yes,2209,0,0,no,lived



Jeg kan se at den data jeg har at arbejde med er labelled data. Labelled er 'outcome', hvor resten bortset fra id, er features som har en indflydelse på outcome. 

### Drop unødvendig data kolonner


In [4]:
if data.get('id') is not None:
    data = data.drop('id', axis=1)

#data = data.drop('hospital_number', axis=1)

data = data.drop('lesion_2', axis=1)
data = data.drop('lesion_3', axis=1)

# Manglende værdier

In [5]:
print(data.isna().sum())

surgery                    0
age                        0
hospital_number            0
rectal_temp                0
pulse                      0
respiratory_rate           0
temp_of_extremities       39
peripheral_pulse          60
mucous_membrane           21
capillary_refill_time      6
pain                      44
peristalsis               20
abdominal_distention      23
nasogastric_tube          80
nasogastric_reflux        21
nasogastric_reflux_ph      0
rectal_exam_feces        190
abdomen                  213
packed_cell_volume         0
total_protein              0
abdomo_appearance         48
abdomo_protein             0
surgical_lesion            0
lesion_1                   0
cp_data                    0
outcome                    0
dtype: int64


Her kan jeg se hvor mange manglende værdier der er for hver feature. Dem skal jeg have gjort noget ved.

In [6]:
categorical_columns = ['surgery', 'age','surgical_lesion','cp_data', 'temp_of_extremities','peripheral_pulse','mucous_membrane','capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux', 'rectal_exam_feces', 'abdomen', 'abdomo_appearance', 'outcome']

In [7]:
encoder = OrdinalEncoder()

In [8]:
def encode(data_to_encode):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data_to_encode.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data_to_encode.loc[data_to_encode.notnull()] = np.squeeze(impute_ordinal)
    return data_to_encode

#create a for loop to iterate through each column in the data
for columns in categorical_columns:
    encode(data[columns])

C:\Users\chris\AppData\Local\Temp\ipykernel_21492\421248043.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_encode.loc[data_to_encode.notnull()] = np.squeeze(impute_ordinal)
C:\Users\chris\AppData\Local\Temp\ipykernel_21492\421248043.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_encode.loc[data_to_encode.notnull()] = np.squeeze(impute_ordinal)
C:\Users\chris\AppData\Local\Temp\ipykernel_21492\421248043.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [9]:
#imputer = IterativeImputer(
    #RandomForestRegressor(random_state=42),
    #BayesianRidge(),
    #KNeighborsRegressor(n_neighbors=15),
    #ExtraTreesRegressor(random_state=42)
    #random_state=42, 
    #max_iter=25)
    
#imputer = KNNImputer()

imputer = SimpleImputer(missing_values=np.nan, strategy="mean", add_indicator=False)

In [10]:
data

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,cp_data,outcome
0,1.0,0.0,530001,38.1,132.0,24.0,1.0,3.0,2.0,2.0,1.0,0.0,3.0,2.0,0.0,6.5,1.0,1.0,57.0,8.5,2.0,3.4,1.0,2209,0.0,0.0
1,1.0,0.0,533836,37.5,88.0,12.0,1.0,2.0,4.0,2.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,33.0,64.0,2.0,2.0,1.0,2208,0.0,1.0
2,1.0,0.0,529812,38.3,120.0,28.0,1.0,3.0,5.0,1.0,2.0,3.0,0.0,2.0,2.0,3.5,NaN,0.0,37.0,6.4,2.0,3.4,1.0,5124,0.0,2.0
3,1.0,0.0,5262541,37.1,72.0,30.0,0.0,3.0,5.0,2.0,3.0,3.0,0.0,2.0,1.0,2.0,1.0,1.0,53.0,7.0,1.0,3.9,1.0,2208,1.0,2.0
4,0.0,0.0,5299629,38.0,52.0,48.0,2.0,2.0,3.0,1.0,0.0,3.0,1.0,2.0,0.0,7.0,3.0,3.0,47.0,7.3,1.0,2.6,0.0,0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1.0,0.0,535246,38.5,129.0,48.0,1.0,3.0,5.0,2.0,1.0,0.0,0.0,0.0,1.0,2.0,NaN,0.0,57.0,66.0,2.0,2.0,1.0,2206,0.0,2.0
1231,1.0,0.0,528570,37.5,60.0,50.0,1.0,3.0,4.0,1.0,3.0,3.0,3.0,2.0,2.0,3.0,1.0,1.0,35.0,6.4,2.0,3.6,1.0,2209,1.0,0.0
1232,1.0,1.0,529685,37.5,84.0,40.0,2.0,3.0,3.0,1.0,3.0,3.0,3.0,2.0,2.0,3.0,2.0,2.0,40.0,5.9,1.0,7.0,1.0,400,1.0,2.0
1233,1.0,0.0,534784,38.1,70.0,16.0,2.0,3.0,1.0,1.0,3.0,3.0,3.0,0.0,1.0,2.0,0.0,1.0,58.0,74.0,1.0,2.0,1.0,2209,0.0,2.0


In [12]:
# impute data and convert 
data[categorical_columns] = pd.DataFrame(np.round(imputer.fit_transform(data[categorical_columns])),columns = categorical_columns)

In [13]:
data

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,cp_data,outcome
0,1.0,0.0,530001,38.1,132.0,24.0,1.0,3.0,2.0,2.0,1.0,0.0,3.0,2.0,0.0,6.5,1.0,1.0,57.0,8.5,2.0,3.4,1.0,2209,0.0,0.0
1,1.0,0.0,533836,37.5,88.0,12.0,1.0,2.0,4.0,2.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,33.0,64.0,2.0,2.0,1.0,2208,0.0,1.0
2,1.0,0.0,529812,38.3,120.0,28.0,1.0,3.0,5.0,1.0,2.0,3.0,0.0,2.0,2.0,3.5,1.0,0.0,37.0,6.4,2.0,3.4,1.0,5124,0.0,2.0
3,1.0,0.0,5262541,37.1,72.0,30.0,0.0,3.0,5.0,2.0,3.0,3.0,0.0,2.0,1.0,2.0,1.0,1.0,53.0,7.0,1.0,3.9,1.0,2208,1.0,2.0
4,0.0,0.0,5299629,38.0,52.0,48.0,2.0,2.0,3.0,1.0,0.0,3.0,1.0,2.0,0.0,7.0,3.0,3.0,47.0,7.3,1.0,2.6,0.0,0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1.0,0.0,535246,38.5,129.0,48.0,1.0,3.0,5.0,2.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,57.0,66.0,2.0,2.0,1.0,2206,0.0,2.0
1231,1.0,0.0,528570,37.5,60.0,50.0,1.0,3.0,4.0,1.0,3.0,3.0,3.0,2.0,2.0,3.0,1.0,1.0,35.0,6.4,2.0,3.6,1.0,2209,1.0,0.0
1232,1.0,1.0,529685,37.5,84.0,40.0,2.0,3.0,3.0,1.0,3.0,3.0,3.0,2.0,2.0,3.0,2.0,2.0,40.0,5.9,1.0,7.0,1.0,400,1.0,2.0
1233,1.0,0.0,534784,38.1,70.0,16.0,2.0,3.0,1.0,1.0,3.0,3.0,3.0,0.0,1.0,2.0,0.0,1.0,58.0,74.0,1.0,2.0,1.0,2209,0.0,2.0


In [14]:
data.isna().sum()

surgery                  0
age                      0
hospital_number          0
rectal_temp              0
pulse                    0
respiratory_rate         0
temp_of_extremities      0
peripheral_pulse         0
mucous_membrane          0
capillary_refill_time    0
pain                     0
peristalsis              0
abdominal_distention     0
nasogastric_tube         0
nasogastric_reflux       0
nasogastric_reflux_ph    0
rectal_exam_feces        0
abdomen                  0
packed_cell_volume       0
total_protein            0
abdomo_appearance        0
abdomo_protein           0
surgical_lesion          0
lesion_1                 0
cp_data                  0
outcome                  0
dtype: int64

### Gem i ny csv

In [15]:
data.to_csv('data/train_cleaned.csv', index=False)